# **JORNADA DE DADOS <Laboratória>**
# **PROJ005**

A VAREJISTA, empresa líder no setor, enfrenta o desafio de gerenciar grandes volumes de dados dispersos. Para aprimorar a tomada de decisões, está sendo implementado um sistema ETL robusto com tabelas fato e dimensões. O projeto visa extrair, transformar e carregar dados de forma eficiente, estruturando-os hierarquicamente para facilitar análises estratégicas. Além de otimizar o armazenamento, a iniciativa busca ampliar a capacidade da empresa de identificar padrões e oportunidades de mercado, respondendo com agilidade às mudanças na demanda do consumidor.
<br><br>
• [**DATASET**](https://github.com/4L4N4F/Python_ETLpipeline/blob/main/superstore.csv)
<br><br>
• **DESCRIÇÃO DAS VARIÁVEIS** <small>(Conforme fornecidas pela Laboratória na documentação do projeto.)</small>
<br><br>
| Variável       | Descrição                                                                        | Tipo      |
|----------------|----------------------------------------------------------------------------------|-----------|
| category       | Categoria do produto adquirido                                                   | STRING    |
| city           | Cidade onde o pedido foi feito                                                   | STRING    |
| country        | País onde o hipermercado está localizado                                         | STRING    |
| customer_id    | Identificador único para cada cliente                                             | STRING    |
| customer_name  | Nome do cliente                                                                  | STRING    |
| discount       | Desconto aplicado no pedido                                                      | FLOAT     |
| market         | Mercado ou região onde o hipermercado atua                                        | STRING    |
| unknown        | Coluna desconhecida ou não especificada                                           | INTEGER   |
| order_date     | Data do pedido                                                                    | TIMESTAMP |
| order_id       | Identificador único para cada pedido                                             | STRING    |
| order_priority | Nível de prioridade do pedido                                                     | STRING    |
| product_id     | Identificador exclusivo para cada produto                                        | STRING    |
| product_name   | Nome do produto                                                                  | STRING    |
| profit         | Lucro gerado pelo pedido                                                          | FLOAT     |
| quantity       | Quantidade de produtos encomendados no pedido                                    | INTEGER   |
| region         | Região onde o pedido foi feito                                                   | STRING    |
| row_id         | Identificador exclusivo para cada linha do conjunto de dados                      | INTEGER   |
| sales          | Valor total de vendas do produto no pedido                                       | INTEGER   |
| segment        | Segmento do cliente                                                              | STRING    |
| ship_date      | Data de envio do pedido                                                           | TIMESTAMP |
| ship_mode      | Modo de envio do pedido                                                          | STRING    |
| shipping_cost  | Custo de envio do pedido                                                         | FLOAT     |
| state          | Estado ou região do país                                                         | STRING    |
| sub_category   | Subcategoria do produto                                                          | STRING    |
| year           | Ano do pedido                                                                    | INTEGER   |
| market2        | Outra coluna relacionada a informações de mercado                               | STRING    |
| weeknum        | Número da semana em que o pedido foi feito                                       | INTEGER   |

<br><br>
• **ETAPAS:**<br>
1. Limpeza (df_limpo)<br>
2. Web scraping (df_multinacional_limpo)<br>
3. Construção de dimensões + fatos<br>
4. Preparação para upload (staging / merge)<br>
5. Upload para BigQuery

In [59]:
# SETUP

import unicodedata
import pandas as pd
import numpy as np
import re
from urllib.request import Request, urlopen
from urllib.error import URLError, HTTPError
from bs4 import BeautifulSoup

df = pd.read_csv('https://github.com/4L4N4F/Python_ETLpipeline/raw/refs/heads/main/superstore.csv')

### Etapa 1. Limpeza

In [60]:
# nulos
if df.isnull().values.any():
    print("Há valores nulos no DataFrame.")
else:
    print("Não há valores nulos no DataFrame.")

#duplicatas
if df.duplicated().any():
    print("Há linhas duplicadas no DataFrame.")
else:
    print("Não há linhas duplicadas no DataFrame.")

Não há valores nulos no DataFrame.
Não há linhas duplicadas no DataFrame.


In [61]:
# definir as variáveis categóricas
df.columns = df.columns.str.lower()
cat_cols = df.select_dtypes(include='object').columns
print("variáveis:", cat_cols)

# não agir em colunas que devem ser id ou nome
df.columns = df.columns.str.lower()
cols_to_clean = [c for c in cat_cols if 'id' not in c and 'name' not in c and 'date' not in c]
print(cols_to_clean)



variáveis: Index(['category', 'city', 'country', 'customer_id', 'customer_name', 'market',
       'order_date', 'order_id', 'order_priority', 'product_id',
       'product_name', 'region', 'segment', 'ship_date', 'ship_mode', 'state',
       'sub_category', 'market2'],
      dtype='object')
['category', 'city', 'country', 'market', 'order_priority', 'region', 'segment', 'ship_mode', 'state', 'sub_category', 'market2']


In [62]:
# padronização das categóricas
df_limpo = df.copy()
def padroniza_cols_to_clean(df, cols_to_clean):
    def remove_acentos(texto):
        if isinstance(texto, str):
            texto = unicodedata.normalize('NFKD', texto)
            texto = ''.join(c for c in texto if not unicodedata.combining(c))
        return texto
    for col in cols_to_clean:
        df[col] = df[col].astype(str).str.strip().str.lower().apply(remove_acentos)

    return df

# aplica função à cópia do df
df_limpo = padroniza_cols_to_clean(df_limpo, cols_to_clean)

# verificação sample
df_limpo[cols_to_clean].sample(10, random_state=42)



,category,city,country,market,order_priority,region,segment,ship_mode,state,sub_category,market2
49728,technology,philadelphia,united states,us,medium,east,consumer,second class,pennsylvania,accessories,north america
45547,office supplies,wollongong,australia,apac,medium,oceania,corporate,second class,new south wales,supplies,apac
15664,office supplies,galati,romania,emea,medium,emea,corporate,standard class,galati,binders,emea
40561,office supplies,lisbon,portugal,eu,medium,south,consumer,standard class,lisboa,paper,eu
49426,furniture,coral springs,united states,us,high,south,corporate,same day,florida,furnishings,north america
18956,office supplies,prague,czech republic,emea,medium,emea,consumer,standard class,prague,binders,emea
14238,office supplies,vienna,austria,emea,medium,emea,corporate,standard class,vienna,envelopes,emea
31818,office supplies,lagos,nigeria,africa,medium,africa,corporate,first class,lagos,art,africa
41388,furniture,buenos aires,argentina,latam,medium,south,consumer,standard class,buenos aires,chairs,latam
37236,office supplies,la ceiba,honduras,latam,high,central,corporate,standard class,atlantida,storage,latam


In [63]:
# verificações das variáveis numéricas

num_cols = df_limpo.select_dtypes(include=['int64', 'float64']).columns

outliers_dict = {}
nulls_dict = {}
invalid_dict = {}

for col in num_cols:
    # nulos
    nulls_dict[col] = df_limpo[col].isnull().sum()

    # inválidos
    invalid_values = df_limpo[~df_limpo[col].apply(lambda x: np.isreal(x)) & df_limpo[col].notnull()]
    invalid_dict[col] = len(invalid_values)

    # outliers
    q1 = df_limpo[col].quantile(0.25)
    q3 = df_limpo[col].quantile(0.75)
    iqr = q3 - q1
    lower, upper = q1 - 1.5*iqr, q3 + 1.5*iqr
    outliers = df_limpo[(df_limpo[col] < lower) | (df_limpo[col] > upper)]

    outliers_dict[col] = len(outliers)

variable_report = pd.DataFrame({
    'var': num_cols,
    'null': [nulls_dict[c] for c in num_cols],
    'inv': [invalid_dict[c] for c in num_cols],
    'outliers': [outliers_dict[c] for c in num_cols]
})

variable_report

,var,null,inv,outliers
0,discount,0,0,4172
1,unknown,0,0,0
2,profit,0,0,9755
3,quantity,0,0,877
4,row_id,0,0,0
5,sales,0,0,5655
6,shipping_cost,0,0,5909
7,year,0,0,0
8,weeknum,0,0,0


###Etapa 2. Web scraping

In [64]:
wiki = "https://en.wikipedia.org/wiki/List_of_supermarket_chains"
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 '
                  '(KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

try:
    req = Request(wiki, headers=headers)
    wiki_page = urlopen(req)
    soup = BeautifulSoup(wiki_page, 'html.parser')
    wiki_page.close()

    tabela = soup.find('table', class_='wikitable')
    if tabela is None:
        raise ValueError("tabela não encontrada")

    linhas = tabela.find_all('tr')
    cabecalho = [th.text.strip() for th in linhas[0].find_all('th')]

    dados = []
    for linha in linhas[1:]:
        colunas = linha.find_all(['td', 'th'])
        if len(colunas) > 0:
            dados.append([coluna.text.strip() for coluna in colunas])

    df_multinacional = pd.DataFrame(dados, columns=cabecalho)

    df_multinacional.columns = df_multinacional.columns.str.lower()
    df_multinacional = df_multinacional.drop(columns=['map'], errors='ignore')
    df_multinacional = df_multinacional.rename(columns={
        "served countries (besides the headquarters)": "countries",
        "number of locations": "locations",
        "number of employees": "employees"
    })

    def extract_number_clean(value):
        if pd.isna(value) or str(value).strip() == '':
            return None

        # remove vírgulas e pontos
        value_clean = str(value).replace(',', '').replace('.', '')
        match = re.search(r'\d+', value_clean)
        return int(match.group()) if match else None

    df_multinacional['locations'] = df_multinacional['locations'].apply(extract_number_clean)
    df_multinacional['employees'] = df_multinacional['employees'].apply(extract_number_clean)


    def count_countries(value):
        if pd.isna(value) or str(value).strip() == '':
            return 1
        return len([c.strip() for c in str(value).split(',') if c.strip() and c.strip() != 'nan'])

    df_multinacional['countries_count'] = df_multinacional['countries'].apply(count_countries)

    # abre cópia limpa e padroniza strings
    df_multinacional_limpo = df_multinacional.copy()
    for col in df_multinacional_limpo.select_dtypes(include='object').columns:
        df_multinacional_limpo[col] = df_multinacional_limpo[col].str.strip().str.lower()

    # verificação
    display(df_multinacional_limpo.head(10))

except (URLError, HTTPError) as e:
    print("Erro ao acessar a página:", e)
except Exception as e:
    print("Erro no processamento:", e)

,company,headquarters,countries,locations,employees,countries_count
0,7-eleven,japanunited states,"australia, canada, china, cambodia, denmark, h...",85000.0,170000.0,17
1,aeon,japan,"australia, cambodia, china, hong kong, india, ...",20008.0,560000.0,15
2,ahold delhaize,netherlands,"belgium (as albert heijn and delhaize), czech ...",7659.0,375000.0,14
3,aldi,germany (süd and nord),"australia (süd), austria (süd as hofer), china...",12832.0,274172.0,18
4,edeka,germany,denmark,13646.0,381000.0,1
5,extra,germany,,NaN,9700.0,1
6,minimal,germany,poland,1500.0,NaN,1
7,auchan,france,"spain, luxembourg, italy, portugal, poland, ro...",2101.0,160407.0,15
8,big c,thailand,"laos, vietnam",1234.0,27000.0,2
9,billa,austria,"bulgaria, czech republic, slovakia",3645.0,76174.0,3


###Etapa 3. Construção de dimensões + fatos